In [1]:
import pandas

import numpy as np # importar la librería numpy

import requests # importar la librería requests

from selenium import webdriver # Habrá que descargar el webdriver de chrome para realizar las distintas operaciones

from selenium.webdriver.support.ui import WebDriverWait # Cuando cargamos una página web a veces el HTML tarda un poco en cargarse sugún nuestra velocidad de conexión pues hasta ciertos elementos no carguen tendremos que esperar

from selenium.webdriver.support import expected_conditions as EC

from selenium.webdriver.common.by import By

from selenium.webdriver.support.ui import Select

from selenium.webdriver.common.keys import Keys

import time

import os

from geopy.geocoders import Nominatim

import time

import re

import math

In [2]:
def pot_electro():
    options = webdriver.ChromeOptions()
    
    options.add_argument("--start-maximized")
    
    options.add_argument("--disable-extensions")
    
    driver_path = '/Users/mohamedelmarraki/ironhack/Final-Project/chromedriver'
    
    driver = webdriver.Chrome(driver_path, chrome_options = options)
    
    driver.get("https://www.iberdrola.es/blog/energia/cual-es-la-potencia-necesaria-para-los-electrodomesticos")


    potencia_electro1 = driver.find_element_by_xpath("/html/body/div[1]/main/div[4]/div[1]/div[3]/ul[2]")
    potencia_electro1 = potencia_electro1.text

    time.sleep(1)
    
    driver.close()


    options = webdriver.ChromeOptions()
    
    options.add_argument("--start-maximized")
    
    options.add_argument("--disable-extensions")
    
    driver_path = '/Users/mohamedelmarraki/ironhack/Final-Project/chromedriver'
    
    driver = webdriver.Chrome(driver_path, chrome_options = options)
    
    driver.get("http://www.bajatelapotencia.org/la-potencia-que-necesitas/")


    WebDriverWait(driver, 5)\
            .until(EC.element_to_be_clickable((By.XPATH, "/html/body/div[2]/section/article/table/tbody")))


    potencia_electro2 = driver.find_element_by_xpath("/html/body/div[2]/section/article/table/tbody")
    potencia_electro2 = potencia_electro2.text

    time.sleep(1)
    
    driver.close()


    options = webdriver.ChromeOptions()
    
    options.add_argument("--start-maximized")
    
    options.add_argument("--disable-extensions")
    
    driver_path = '/Users/mohamedelmarraki/ironhack/Final-Project/chromedriver'
    
    driver = webdriver.Chrome(driver_path, chrome_options = options)
    
    driver.get("https://www.electricaplicada.com/potencia-consumo-equipos-electricos/")


    WebDriverWait(driver, 5)\
            .until(EC.element_to_be_clickable((By.XPATH, "//*[@id='post-4887']/div/div")))


    potencia_electro3 = driver.find_element_by_xpath("//*[@id='post-4887']/div/div")
    potencia_electro3 = potencia_electro3.text

    time.sleep(1)
    
    driver.close()
    
    potencia_electro1 = potencia_electro1.split("\n")
    potencia_electro2 = potencia_electro2.split("\n")
    potencia_electro2 = potencia_electro2[7:-1]

    x = re.split("esenciales.", potencia_electro3, 1)
    x = x[1]
    x = x.split("\n")
    lista = []
    for i in x:
        if re.findall("[1-9]", i):
            lista.append(i)
    lista2 = []
    for i in lista:
        if "–" not in i and "*" not in i and  "/" not in i and  "(" not in i and "-" not in i:
            lista2.append(i)
    potencia_electro3 = lista2[:-2]
    potencia_electro3[0] = "Licuadora 500W"
    potencia_electro1_list1 = []
    for i in potencia_electro1:
        i = i.replace("-", "")
        i = i.replace(":", "")
        i = i.replace(".", "")
        i = i.replace("  ", " ")
        potencia_electro1_list1.append(re.split('\s|(?<!\d)[,.](?!\d)', i))
    potencia_electro1_list2 = []
    for i in potencia_electro1_list1:
        if i[0] == 'Aire':
            i[0] = 'Aire acondicionado'
            del i[1]
        potencia_electro1_list2.append(i)
   
    df = pandas.DataFrame(potencia_electro1_list2)
    df.drop([3], axis=1, inplace=True)
    
    potencia2 = []
    for i in potencia_electro2:
        i = i.split("(")
        potencia2.append(i)
    potencia2a = []
    for i in potencia2:
        potencia2a.append(re.split('\s|(?<!\d)[,.](?!\d)', i[0]))
    potencia2b = []
    for i in potencia2a:
        if i[0] == 'Aire':
            i[0] = 'Aire acondicionado'
            del i[1]
        elif i[0] == "Calefacción":
            i[0] = 'Calefacción eléctrica'
            del i[1]
        potencia2b.append(i)
    
    
    df1 = pandas.DataFrame(potencia2b)
    df1.drop([2, 5], axis=1, inplace=True)
    df1.rename(columns={3:2, 4:3}, inplace=True)
    df1.drop([3], axis=1, inplace=True) 
    
    
    df_conc = pandas.concat([df, df1], axis=0)
    df_conc.drop([5],axis=0, inplace=True)
    df_conc[[1, 2]] = df_conc[[1, 2]].apply(pandas.to_numeric)
    df_conc[3] = df_conc[[1, 2]].median(axis=1)
    df_conc.drop([1,2], axis=1, inplace=True)
    
    
    lista3 = []
    for i in potencia_electro3:
        lista3.append(list(i.rpartition(" ")))
    
    df3 = pandas.DataFrame(lista3)
    df3 = df3.iloc[[0, 2, 7, 18, 23, 42]]
    lista4 = []
    for i in df3[2]:
        lista4.append(i.replace("W", ""))
    df3[3] = lista4
    df3.drop([1, 2], axis=1, inplace=True)
    df_conc2 = pandas.concat([df_conc, df3], axis=0)
    df_conc2.reset_index(inplace=True)
    df_conc2.drop("index", axis=1, inplace=True)
    df_conc2 = df_conc2.rename(columns={0: 'Electrodomesticos', 3: 'Potencia'})
    df_conc2["Potencia"] = df_conc2["Potencia"].apply(pandas.to_numeric)
    
    
    pot_electrodomesticos = {k: v for k, v in zip(list(df_conc2["Electrodomesticos"]), list(df_conc2["Potencia"]))}
    return pot_electrodomesticos
    
    
    
    
    

In [3]:
pot_electrodomesticos = pot_electro()

/Users/mohamedelmarraki/miniconda3/envs/Project-Final/lib/python3.7/site-packages/ipykernel_launcher.py:10: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  # Remove the CWD from sys.path while we load stuff.
/Users/mohamedelmarraki/miniconda3/envs/Project-Final/lib/python3.7/site-packages/ipykernel_launcher.py:10: DeprecationWarning: use options instead of chrome_options
  # Remove the CWD from sys.path while we load stuff.
/Users/mohamedelmarraki/miniconda3/envs/Project-Final/lib/python3.7/site-packages/ipykernel_launcher.py:15: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  from ipykernel import kernelapp as app
/Users/mohamedelmarraki/miniconda3/envs/Project-Final/lib/python3.7/site-packages/ipykernel_launcher.py:31: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
/Users/mohamedelmarraki/miniconda3/envs/Project-Final/lib/python3.7/

In [4]:
electrodomesticos = list(pot_electrodomesticos.keys())

In [5]:
def electro_usuario():
    print("Previsión de Energía consumida")
    print("Si quiere salir del programa introduce la palabra 'salir'")
    print("\n")
    horas_func = {}
    unidades = {}
    dias_func = {}
    for i in range(len(electrodomesticos)):
        nueva = input(f"¿ Tiene usted {electrodomesticos[i]} en su casa (Sí/No) ? ")
        if nueva.lower() in ["si".lower(), "sí".lower()]:
            unidad = input(f'¿ Cúantas unidades de {electrodomesticos[i]} tiene usted en su casa ? ')
            dias = input(f"¿ Introduce cuantos días a la semana se va utilizar el(a) {electrodomesticos[i]} en su casa ?")
            horas = input(f"Introduce un estimación aproximada de las horas de funcionamiento que va a tener el(a) {electrodomesticos[i]} en los días que esté funcionando : ")
            horas_func[electrodomesticos[i]]=horas
            unidades[electrodomesticos[i]] = unidad
            dias_func[electrodomesticos[i]] = dias       
        elif nueva.lower() == "salir":
            break
        elif dias.lower() == "salir":
            break
        elif horas.lower() == "salir":
            break
        elif unidad.lower() == "salir":
            break
    return horas_func, unidades, dias_func

In [6]:
diccionarios = electro_usuario()
horas_func = diccionarios[0]
unidades = diccionarios[1]
dias_func = diccionarios[2]

Previsión de Energía consumida
Si quiere salir del programa introduce la palabra 'salir'


¿ Tiene usted Frigorífico en su casa (Sí/No) ? si
¿ Cúantas unidades de Frigorífico tiene usted en su casa ? 1
¿ Introduce cuantos días a la semana se va utilizar el(a) Frigorífico en su casa ?7
Introduce un estimación aproximada de las horas de funcionamiento que va a tener el(a) Frigorífico en los días que esté funcionando : 24
¿ Tiene usted Lavadora en su casa (Sí/No) ? si
¿ Cúantas unidades de Lavadora tiene usted en su casa ? 1
¿ Introduce cuantos días a la semana se va utilizar el(a) Lavadora en su casa ?3
Introduce un estimación aproximada de las horas de funcionamiento que va a tener el(a) Lavadora en los días que esté funcionando : 3
¿ Tiene usted Lavavajillas en su casa (Sí/No) ? si
¿ Cúantas unidades de Lavavajillas tiene usted en su casa ? 1
¿ Introduce cuantos días a la semana se va utilizar el(a) Lavavajillas en su casa ?4
Introduce un estimación aproximada de las horas de funcionam

In [11]:
import numpy
import pandas as pd
lista = []
for i in horas_func.keys():
    lista.append(i)



potencia = dict(filter(lambda x: x[0] in lista, pot_electrodomesticos.items()))



from itertools import chain
from collections import defaultdict

dict3 = defaultdict(list)
for k, v in chain(horas_func.items(), unidades.items(), potencia.items(), dias_func.items()):
    dict3[k].append(int(v))
dic_final = dict(dict3)




dic_final2 = {}
lis = []
for i, y in dic_final.items():
    dic_final2[i] = numpy.prod(y)
for i in dic_final2.values():
    a = (i/7)
    lis.append(a)
    print(a)


    
consumo_teorico = sum(lis)





consumo_diario= consumo_teorico/(0.95*0.95*0.9)





suma_de_potencia = 0
for i in potencia.values():
    suma_de_potencia  += i
Potencia_Prevista = suma_de_potencia*0.75
if Potencia_Prevista > 10000:
    print("la instalación necesitará Proyecto de un Ingeniero")
else:
    print("la instalación necesitará una Memoria Técnica de Diseño (MTD) de un Técnico electricista.")
    

    
if Potencia_Prevista < 1500:
    tensión = 12
elif 1500 <= Potencia_Prevista < 5000:
    tensión = 48
elif Potencia_Prevista > 5000:
    tensión = 120
    
    
    
df = pd.read_csv("../../../mohamedelmarraki/Downloads/Monthlydata_40.400_-3.674_SA2_2020_2020.csv", sep="\t", header=[4]).head(12)
df = df.loc[:, ["year", "month","H(i_opt)_m", "T2m"]]
irrandiacia_zona = ((df["H(i_opt)_m"].min()) * 1000)/31
HSP = irrandiacia_zona / 1000
energía_producirá_día = HSP * 450 * 0.9
Nº_paneles = consumo_diario / energía_producirá_día
round(Nº_paneles)

7200.0
2571.4285714285716
2285.714285714286
1714.2857142857142
72000.0
la instalación necesitará una Memoria Técnica de Diseño (MTD) de un Técnico electricista.


22

In [ ]:
electrodomesticos = '''Aire Acondicionado
Aspiradora
Batidora
Cafetera Eléctrica
Campana Extractora
Horno
Computadoras de Escritorio
Consola de Videojuegos
Estufa
Freidora
Lavadora
Lavavajillas
Licuadora
Microondas
Vitrocerámica
Plancha de Ropa
Refrigerador
Secador de Pelo
Televisor
Tostadora
Lamparas'''
electrodomesticos = electrodomesticos.split("\n")
pot_electrodomesticos = {'Aire Acondicionado': 875,
 'Aspiradora': 450,
 'Batidora': 200,
 'Cafetera Eléctrica': 800,
 'Campana Extractora': 200,
 'Cepillo eléctrico': 55,
 'Computadoras de Escritorio': 220,
 'Consola de Videojuegos': 90,
 'Depiladora Eléctrica': 20,
 'Equipos de Música': 80,
 'Estufa': 2000,
 'Freidora': 1100,
 'Horno' : 1400,
 'Hervidor Eléctrico de Agua': 1200,
 'Lavadora': 1500,
 'Lavavajillas': 1800,
 'Licuadora': 1200,
 'Microondas': 900,
 'Plancha de Pelo': 2000,
 'Plancha de Ropa': 1200,
 'Refrigerador': 300,
 'Sandwichera': 800,
 'Secador de Pelo': 1900,
 'Televisor': 300,
 'Tostadora': 1000,
 'Ventilador': 60,
 'Lamparas' : 60 }

In [ ]:
print("Previsión de Energía consumida")
print("Si quiere salir del programa introduce la palabra 'salir'")
horas_func = {}
unidades = {}
dias_func = {}
for i in range(len(electrodomesticos)):
    nueva = input(f"¿ Tiene usted {electrodomesticos[i]} en su casa (Sí/No) ? ")
    if nueva.lower() in ["si".lower(), "sí".lower()]:
        unidad = input(f'¿ Cúantas unidades de {electrodomesticos[i]} tiene usted en su casa ? ')
        dias = input(f"¿ Introduce cuantos días a la semana se va utilizar el(a) {electrodomesticos[i]} en su casa ?")
        horas = input(f"Introduce un estimación aproximada de las horas de funcionamiento que va a tener el(a) {electrodomesticos[i]} en los días que esté funcionando : ()")
        horas_func[electrodomesticos[i]]=horas
        unidades[electrodomesticos[i]] = unidad
        dias_func[electrodomesticos[i]] = dias       
    elif nueva.lower() == "salir":
        break
                     

In [ ]:
lista = []
for i in horas_func.keys():
    lista.append(i)



potencia = dict(filter(lambda x: x[0] in lista, pot_electrodomesticos.items()))



from itertools import chain
from collections import defaultdict

dict3 = defaultdict(list)
for k, v in chain(horas_func.items(), unidades.items(), potencia.items(), dias_func.items()):
    dict3[k].append(int(v))
dic_final = dict(dict3)




dic_final2 = {}
lis = []
for i, y in dic_final.items():
    dic_final2[i] = numpy.prod(y)
for i in dic_final2.values():
    a = (i/7)
    lis.append(a)
    print(a)


    
consumo_teorico = sum(lis)





consumo_diario= consumo_teorico/(0.95*0.95*0.9)





suma_de_potencia = 0
for i in potencia.values():
    suma_de_potencia  += i
Potencia_Prevista = suma_de_potencia*0.75
if Potencia_Prevista > 10000:
    print("la instalación necesitará Proyecto de un Ingeniero")
else:
    print("la instalación necesitará una Memoria Técnica de Diseño (MTD) de un Técnico electricista.")
    

    
if Potencia_Prevista < 1500:
    tensión = 12
elif 1500 <= Potencia_Prevista < 5000:
    tensión = 48
elif Potencia_Prevista > 5000:
    tensión = 120
    
    
    
df = pd.read_csv("../../../mohamedelmarraki/Downloads/Monthlydata_40.000_-3.000_SA2_2020_2020.csv", sep="\t", header=[4]).head(12)
df = df.loc[:, ["year", "month","H(i_opt)_m", "T2m"]]
irrandiacia_zona = ((df["H(i_opt)_m"].min()) * 1000)/31
HSP = irrandiacia_zona / 1000
energía_producirá_día = HSP * 450 * 0.9
Nº_paneles = 29999.084465549837 / energía_producirá_día
round(Nº_paneles)

In [ ]:
lista = []
for i in horas_func.keys():
    lista.append(i)
lista

### los componentes de casa con su potencia

In [ ]:
potencia = dict(filter(lambda x: x[0] in lista, pot_electrodomesticos.items()))
potencia 

### los componentes de casa con sus días de funcionamiento

In [ ]:
dias_func

### los componentes de casa con sus días de funcionamiento

In [ ]:
horas_func

### los componentes de casa con toda su información

In [ ]:
from itertools import chain
from collections import defaultdict

dict3 = defaultdict(list)
for k, v in chain(horas_func.items(), unidades.items(), potencia.items(), dias_func.items()):
    dict3[k].append(int(v))
dic_final = dict(dict3)
dic_final

### los componentes de casa con su energía consumida

In [ ]:
dic_final2 = {}
lis = []
for i, y in dic_final.items():
    dic_final2[i] = numpy.prod(y)
for i in dic_final2.values():
    a = (i/7)
    lis.append(a)
    print(a)

### consumo teorico `Lmd`

In [ ]:
consumo_teorico = sum(lis)
consumo_teorico

### Consumo Real

In [ ]:
consumo_diario= consumo_teorico/(0.95*0.95*0.9)
consumo_diario

### Previsión de Potencia por Receptores

In [ ]:
suma_de_potencia = 0
for i in potencia.values():
    suma_de_potencia  += i
Potencia_Prevista = suma_de_potencia*0.75
if Potencia_Prevista > 10000:
    print("la instalación necesitará Proyecto de un Ingeniero")
else:
    print("la instalación necesitará una Memoria Técnica de Diseño (MTD) de un Técnico electricista.")

In [ ]:
Potencia_Prevista

In [ ]:
if Potencia_Prevista < 1500:
    tensión = 12
elif 1500 <= Potencia_Prevista < 5000:
    tensión = 48
elif Potencia_Prevista > 5000:
    tensión = 120

In [ ]:
import os
import pandas as pd

In [ ]:
df = pd.read_csv("../../../mohamedelmarraki/Downloads/Monthlydata_40.000_-3.000_SA2_2020_2020.csv", sep="\t", header=[4]).head(12)
df = df.loc[:, ["year", "month","H(i_opt)_m", "T2m"]]
irrandiacia_zona = ((df["H(i_opt)_m"].min()) * 1000)/31
HSP = irrandiacia_zona / 1000
energía_producirá_día = HSP * 450 * 0.9
Nº_paneles = 29999.084465549837 / energía_producirá_día
round(Nº_paneles)




In [ ]:
electro_usuario()

In [ ]:
import pandas as pd # importar la librería pandas

import numpy as np # importar la librería numpy

import requests # importar la librería requests

from selenium import webdriver # Habrá que descargar el webdriver de chrome para realizar las distintas operaciones

from selenium.webdriver.support.ui import WebDriverWait # Cuando cargamos una página web a veces el HTML tarda un poco en cargarse sugún nuestra velocidad de conexión pues hasta ciertos elementos no carguen tendremos que esperar

from selenium.webdriver.support import expected_conditions as EC

from selenium.webdriver.common.by import By

from selenium.webdriver.support.ui import Select

from selenium.webdriver.common.keys import Keys

import time

import os

from geopy.geocoders import Nominatim

import time

import math

In [ ]:
options = webdriver.ChromeOptions()
    
options.add_argument("--start-maximized")
    
options.add_argument("--disable-extensions")
    
driver_path = '/Users/mohamedelmarraki/ironhack/Final-Project/chromedriver'
    
driver = webdriver.Chrome(driver_path, chrome_options = options)
    
driver.get("https://www.iberdrola.es/blog/energia/cual-es-la-potencia-necesaria-para-los-electrodomesticos")


potencia_electro1 = driver.find_element_by_xpath("/html/body/div[1]/main/div[4]/div[1]/div[3]/ul[2]")
potencia_electro1 = potencia_electro1.text

time.sleep(1)
    
driver.close()


options = webdriver.ChromeOptions()
    
options.add_argument("--start-maximized")
    
options.add_argument("--disable-extensions")
    
driver_path = '/Users/mohamedelmarraki/ironhack/Final-Project/chromedriver'
    
driver = webdriver.Chrome(driver_path, chrome_options = options)
    
driver.get("http://www.bajatelapotencia.org/la-potencia-que-necesitas/")


WebDriverWait(driver, 5)\
        .until(EC.element_to_be_clickable((By.XPATH, "/html/body/div[2]/section/article/table/tbody")))


potencia_electro2 = driver.find_element_by_xpath("/html/body/div[2]/section/article/table/tbody")
potencia_electro2 = potencia_electro2.text

time.sleep(1)
    
driver.close()


options = webdriver.ChromeOptions()
    
options.add_argument("--start-maximized")
    
options.add_argument("--disable-extensions")
    
driver_path = '/Users/mohamedelmarraki/ironhack/Final-Project/chromedriver'
    
driver = webdriver.Chrome(driver_path, chrome_options = options)
    
driver.get("https://www.electricaplicada.com/potencia-consumo-equipos-electricos/")


WebDriverWait(driver, 5)\
        .until(EC.element_to_be_clickable((By.XPATH, "//*[@id='post-4887']/div/div")))


potencia_electro3 = driver.find_element_by_xpath("//*[@id='post-4887']/div/div")
potencia_electro3 = potencia_electro3.text

time.sleep(1)
    
driver.close()

In [ ]:
potencia_electro1 = potencia_electro1.split("\n")
potencia_electro2 = potencia_electro2.split("\n")
potencia_electro2 = potencia_electro2[7:-1]
import re
x = re.split("esenciales.", potencia_electro3, 1)
x = x[1]
x = x.split("\n")
lista = []
for i in x:
    if re.findall("[1-9]", i): # En este ejemplo le pedimos que nos devuelve todos los elementos que tengan un caracter del 1 al 8
        lista.append(i)
lista2 = []
for i in lista:
    if "–" not in i and "*" not in i and  "/" not in i and  "(" not in i and "-" not in i:
        lista2.append(i)
potencia_electro3 = lista2[:-2]
potencia_electro3[0] = "Licuadora 500W"



In [ ]:
potencia_electro1
potencia_electro1_list1 = []
for i in potencia_electro1:
    i = i.replace("-", "")
    i = i.replace(":", "")
    i = i.replace(".", "")
    i = i.replace("  ", " ")
    potencia_electro1_list1.append(re.split('\s|(?<!\d)[,.](?!\d)', i))
potencia_electro1_list2 = []
for i in potencia_electro1_list1:
    if i[0] == 'Aire':
        i[0] = 'Aire acondicionado'
        del i[1]
    potencia_electro1_list2.append(i)
   
df = pd.DataFrame(potencia_electro1_list2)
df.drop([3], axis=1, inplace=True)
df

In [ ]:
potencia2 = []
for i in potencia_electro2:
    i = i.split("(")
    potencia2.append(i)
potencia2a = []
for i in potencia2:
    potencia2a.append(re.split('\s|(?<!\d)[,.](?!\d)', i[0]))
potencia2b = []
for i in potencia2a:
    if i[0] == 'Aire':
        i[0] = 'Aire acondicionado'
        del i[1]
    elif i[0] == "Calefacción":
        i[0] = 'Calefacción eléctrica'
        del i[1]
    potencia2b.append(i)
potencia2b
import pandas as pd
df1 = pd.DataFrame(potencia2b)
df1.drop([2, 5], axis=1, inplace=True)
df1.rename(columns={3:2, 4:3}, inplace=True)
df1.drop([3], axis=1, inplace=True)

In [ ]:
df_conc = pd.concat([df, df1], axis=0)
df_conc.drop([5],axis=0, inplace=True)
df_conc[[1, 2]] = df_conc[[1, 2]].apply(pd.to_numeric)
df_conc[3] = df_conc[[1, 2]].median(axis=1)
df_conc.drop([1,2], axis=1, inplace=True)
df_conc

In [ ]:
lista3 = []
for i in potencia_electro3:
    lista3.append(list(i.rpartition(" ")))
    
import pandas as pd
df3 = pd.DataFrame(lista3)
df3 = df3.iloc[[0, 2, 7, 18, 23, 42]]
lista = []
for i in df3[2]:
    lista.append(i.replace("W", ""))
df3[3] = lista
df3.drop([1, 2], axis=1, inplace=True)
df_conc2 = pd.concat([df_conc, df3], axis=0)
df_conc2.reset_index(inplace=True)
df_conc2.drop("index", axis=1, inplace=True)
df_conc2 = df_conc2.rename(columns={0: 'Electrodomesticos', 3: 'Potencia'})
df_conc2["Potencia"] = df_conc2["Potencia"].apply(pd.to_numeric)
df_conc2

In [ ]:
pot_electrodomesticos = {k: v for k, v in zip(list(df_conc2["Electrodomesticos"]), list(df_conc2["Potencia"]))}
pot_electrodomesticos